In [2]:
# !pip install torch_geometric


NameError: name 'torch' is not defined

In [4]:

import torch
import torch_geometric
from torch_geometric.data import Data


import torch
import torch_geometric
from torch_geometric.data import Data

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

import os

from torch.nn import Linear, LayerNorm, ReLU, Dropout
from torch_geometric.nn import ChebConv, NNConv, DeepGCNLayer, GATConv, DenseGCNConv, GCNConv, GraphConv
from torch_geometric.data import Data, DataLoader
import torch_geometric.transforms as T

from sklearn.metrics import roc_auc_score
import scipy.sparse as sp


import warnings
warnings.filterwarnings("ignore")

/home/coean/miniconda3/envs/gnn_tutorials/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print(torch.__version__)

1.12.1


In [6]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)


cpu


In [7]:
# datasets_folder = '/content/drive/MyDrive/Colab Notebooks/ssn/projekt/datasets/elliptic_augmented_dataset/'
datasets_folder = '../datasets/elliptic_augmented_dataset/'

classes_filepath = datasets_folder + 'elliptic_txs_classes.csv'
classes_dataframe = pd.read_csv(classes_filepath, sep=',')
classes_dataframe

,txId,class
0,230425980,unknown
1,5530458,unknown
2,232022460,unknown
3,232438397,2
4,230460314,unknown
...,...,...
276700,-80255194,unknown
276701,-229310245,suspicious
276702,-223754303,unknown
276703,-625924180,suspicious


In [8]:
# initially 2 is licit, 1 is illicit, suspicious is also illicit, unknown is unknown
# after remapping 0 is illicit, 1 is licit, 2 is unknown
# rows with class 0 and 1 will be used for training
# rows with class 2 will be used for prediction


def remap_label(label):
  if label == '1': return 0
  if label == 'suspicious': return 0
  if label == '2': return 1
  return 2

In [9]:
classes_dataframe_after_remap = classes_dataframe.copy()
classes_dataframe_after_remap['class'] = classes_dataframe['class'].map(remap_label)
classes_dataframe_after_remap

,txId,class
0,230425980,2
1,5530458,2
2,232022460,2
3,232438397,1
4,230460314,2
...,...,...
276700,-80255194,2
276701,-229310245,0
276702,-223754303,2
276703,-625924180,0


In [10]:
node_features_filepath = datasets_folder + 'elliptic_txs_features.csv'
node_features_dataframe = pd.read_csv(node_features_filepath, sep=',', header=None)
node_features_dataframe

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.056189,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133295,...,-0.537739,-0.582752,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
1,5530458,1,-0.056193,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133311,...,1.016365,0.702410,-1.033609,-1.032884,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
2,232022460,1,-0.056378,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133972,...,0.731702,0.467009,-1.033609,-1.032884,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128
3,232438397,1,0.043005,0.743641,-0.644127,2.874647,-0.068726,2.304311,3.142835,-0.134902,...,-0.553127,-0.595477,0.204643,0.203652,0.994235,0.099367,-0.135784,0.669253,-0.102834,-0.102128
4,230460314,1,0.294595,-0.131268,-1.185821,0.125213,0.375512,0.188604,-0.101009,-0.134776,...,-0.485972,-0.380435,0.484856,0.546219,0.003514,0.312670,0.369103,1.281230,0.201809,0.202687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276700,-80255194,-1,-0.015471,0.043889,1.522650,-0.058083,-0.046514,-0.020871,-0.101009,-0.134902,...,-0.199222,-0.302816,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
276701,-229310245,-1,-0.056634,-0.141863,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134903,...,-0.553127,-0.570028,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128
276702,-223754303,-1,-0.056336,-0.119014,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134890,...,-0.553127,-0.595477,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
276703,-625924180,-1,-0.052898,-0.109119,-1.185821,-0.131401,-0.068726,-0.167505,0.136346,-0.134233,...,-0.553127,0.447922,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128


In [11]:
node_features_dataframe_with_class = node_features_dataframe.copy()
node_features_dataframe_with_class['class'] = classes_dataframe_after_remap['class']
node_features_dataframe_with_class

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,class
0,230425980,1,-0.056189,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133295,...,-0.582752,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
1,5530458,1,-0.056193,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133311,...,0.702410,-1.033609,-1.032884,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
2,232022460,1,-0.056378,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133972,...,0.467009,-1.033609,-1.032884,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,2
3,232438397,1,0.043005,0.743641,-0.644127,2.874647,-0.068726,2.304311,3.142835,-0.134902,...,-0.595477,0.204643,0.203652,0.994235,0.099367,-0.135784,0.669253,-0.102834,-0.102128,1
4,230460314,1,0.294595,-0.131268,-1.185821,0.125213,0.375512,0.188604,-0.101009,-0.134776,...,-0.380435,0.484856,0.546219,0.003514,0.312670,0.369103,1.281230,0.201809,0.202687,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276700,-80255194,-1,-0.015471,0.043889,1.522650,-0.058083,-0.046514,-0.020871,-0.101009,-0.134902,...,-0.302816,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
276701,-229310245,-1,-0.056634,-0.141863,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134903,...,-0.570028,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,0
276702,-223754303,-1,-0.056336,-0.119014,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134890,...,-0.595477,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
276703,-625924180,-1,-0.052898,-0.109119,-1.185821,-0.131401,-0.068726,-0.167505,0.136346,-0.134233,...,0.447922,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,0


In [12]:
node_features_dataframe_with_class_without_timestep = node_features_dataframe_with_class.drop(columns=[1])
node_features_dataframe_with_class_without_timestep

,0,2,3,4,5,6,7,8,9,10,...,158,159,160,161,162,163,164,165,166,class
0,230425980,-0.056189,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133295,-0.088508,...,-0.582752,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
1,5530458,-0.056193,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133311,-0.088517,...,0.702410,-1.033609,-1.032884,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
2,232022460,-0.056378,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133972,-0.088883,...,0.467009,-1.033609,-1.032884,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,2
3,232438397,0.043005,0.743641,-0.644127,2.874647,-0.068726,2.304311,3.142835,-0.134902,-0.058112,...,-0.595477,0.204643,0.203652,0.994235,0.099367,-0.135784,0.669253,-0.102834,-0.102128,1
4,230460314,0.294595,-0.131268,-1.185821,0.125213,0.375512,0.188604,-0.101009,-0.134776,0.033618,...,-0.380435,0.484856,0.546219,0.003514,0.312670,0.369103,1.281230,0.201809,0.202687,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276700,-80255194,-0.015471,0.043889,1.522650,-0.058083,-0.046514,-0.020871,-0.101009,-0.134902,-0.015154,...,-0.302816,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
276701,-229310245,-0.056634,-0.141863,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134903,-0.089391,...,-0.570028,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,0
276702,-223754303,-0.056336,-0.119014,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134890,-0.088828,...,-0.595477,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
276703,-625924180,-0.052898,-0.109119,-1.185821,-0.131401,-0.068726,-0.167505,0.136346,-0.134233,-0.085575,...,0.447922,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,0


In [13]:
def split_edges_into_train_and_test(edges_dataframe, node_features_with_class):
    edges_amount = len(edges_dataframe)
    step = int(edges_amount * 0.02)    
    
    # this may take a long time
    for i, edge in enumerate(np.array(edges_dataframe)):
      if i % step == 0:
        print(f'{ round(i/ edges_amount * 100)}%')
      # is_start_node_unknown = (node_features_dataframe_with_class_and_remapped_node_id.iloc[[edge[0]]]['class'] == 2).values[0]
      found_start_node = node_features_with_class[node_features_with_class[0] == edge[0]]
    
      #ignore edges for which a node does not exist (this happened once)
      if len(found_start_node) == 0:
        continue
      
      found_end_node = node_features_with_class[node_features_with_class[0] == edge[1]]
      if len(found_end_node) == 0:
        continue
    
      if found_start_node.values[0, -1] == 2:
        test_edges.append(edge)
        continue
      
      if found_end_node.values[0, -1] == 2:
        test_edges.append(edge)
      else:
        train_edges.append(edge)
          
    train_edges = pd.DataFrame(train_edges)
    test_edges = pd.DataFrame(test_edges)
    return train_edges, test_edges

In [14]:
train_edges_filepath = datasets_folder + 'train_edges.csv'
test_edges_filepath = datasets_folder + 'test_edges.csv'

train_edges = [] # edges which have both nodes labeled.
test_edges = [] # edges which one or both nodes are unlabeled.
if os.path.isfile(train_edges_filepath) and os.path.isfile(test_edges_filepath):
    train_edges = pd.read_csv(train_edges_filepath, sep=',').drop(columns=['Unnamed: 0']).rename(columns = {'0': 0, '1':1})
    test_edges = pd.read_csv(test_edges_filepath, sep=',').drop(columns=['Unnamed: 0']).rename(columns = {'0': 0, '1':1})
    
else:
    edges_filepath = datasets_folder + 'elliptic_txs_edgelist.csv'
    edges_dataframe = pd.read_csv(edges_filepath, sep=',')
    train_edges, test_edges = split_edges_into_train_and_test(edges_dataframe, node_features_dataframe_with_class_without_timestep)
    
    # save the train_edges and test_edges to file so that the expensive split does not have to run every time.
    train_edges.to_csv(datasets_folder + 'train_edges.csv')
    test_edges.to_csv(datasets_folder + 'test_edges.csv')

print(len(train_edges))
print(len(test_edges))

train_edges

36633
197768


,0,1
0,232344069,27553029
1,3881097,232457116
2,232051089,232470704
3,230473487,7089694
4,231182296,14660781
...,...,...
36628,-335492958,336712245
36629,-16565051,21440143
36630,-335493020,336712245
36631,-335493021,336712245


,0,1
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206
...,...,...
197763,-1937385,42970508
197764,-12085397,311424426
197765,-16168772,311424426
197766,-357787,355180895


In [15]:
# DONE TODO: fix error when edge at i=234355, edge = array([-573572551,  575769024]) is giving error: IndexError: index 0 is out of bounds for axis 0 with size 0

In [16]:
node_features_dataframe_with_class_without_timestep

,0,2,3,4,5,6,7,8,9,10,...,158,159,160,161,162,163,164,165,166,class
0,230425980,-0.056189,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133295,-0.088508,...,-0.582752,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
1,5530458,-0.056193,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133311,-0.088517,...,0.702410,-1.033609,-1.032884,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
2,232022460,-0.056378,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133972,-0.088883,...,0.467009,-1.033609,-1.032884,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,2
3,232438397,0.043005,0.743641,-0.644127,2.874647,-0.068726,2.304311,3.142835,-0.134902,-0.058112,...,-0.595477,0.204643,0.203652,0.994235,0.099367,-0.135784,0.669253,-0.102834,-0.102128,1
4,230460314,0.294595,-0.131268,-1.185821,0.125213,0.375512,0.188604,-0.101009,-0.134776,0.033618,...,-0.380435,0.484856,0.546219,0.003514,0.312670,0.369103,1.281230,0.201809,0.202687,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276700,-80255194,-0.015471,0.043889,1.522650,-0.058083,-0.046514,-0.020871,-0.101009,-0.134902,-0.015154,...,-0.302816,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
276701,-229310245,-0.056634,-0.141863,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134903,-0.089391,...,-0.570028,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,0
276702,-223754303,-0.056336,-0.119014,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134890,-0.088828,...,-0.595477,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
276703,-625924180,-0.052898,-0.109119,-1.185821,-0.131401,-0.068726,-0.167505,0.136346,-0.134233,-0.085575,...,0.447922,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,0


In [59]:
# Splitting nodes into train and test
train_node_features = node_features_dataframe_with_class_without_timestep[node_features_dataframe_with_class_without_timestep['class'] != 2]
test_node_features = node_features_dataframe_with_class_without_timestep[node_features_dataframe_with_class_without_timestep['class'] == 2]
train_node_features

,0,2,3,4,5,6,7,8,9,10,...,158,159,160,161,162,163,164,165,166,class
3,232438397,0.043005,0.743641,-0.644127,2.874647,-0.068726,2.304311,3.142835,-0.134902,-0.058112,...,-0.595477,0.204643,0.203652,0.994235,0.099367,-0.135784,0.669253,-0.102834,-0.102128,1
9,232029206,-0.006835,0.151139,-0.102433,0.913384,-0.068726,1.026508,0.136346,-0.134902,-0.074868,...,-0.595477,0.204643,0.203652,0.553915,0.014046,-0.135784,0.326888,-0.102834,-0.102128,1
10,232344069,-0.049186,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.108216,-0.074608,...,-0.595477,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,1
11,27553029,-0.050225,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.111938,-0.076671,...,-0.563666,-1.033609,-1.032884,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,1
16,3881097,-0.056437,-0.175567,-1.185821,-0.149731,-0.046514,-0.146557,-0.021890,-0.134896,-0.089198,...,-0.582752,0.204643,0.203652,0.003514,-0.071275,-0.084494,-0.058272,-1.775510,-1.775746,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276695,-439441678,-0.047286,-0.163385,-0.644127,-0.186390,-0.068726,-0.167505,-0.101009,-0.101413,-0.070838,...,-0.582752,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,0
276697,-412642228,-0.026620,-0.118170,0.980956,-0.094742,-0.046514,-0.146557,0.136346,-0.128138,-0.063413,...,-0.582752,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,0
276699,-79327033,-0.053926,-0.175567,-1.185821,-0.186390,-0.068726,-0.167505,-0.101009,-0.125191,-0.084016,...,-0.506406,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,0
276701,-229310245,-0.056634,-0.141863,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134903,-0.089391,...,-0.570028,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,0


In [60]:
test_node_features_without_class = test_node_features.drop(columns=['class'])
test_node_features_without_class

,0,2,3,4,5,6,7,8,9,10,...,157,158,159,160,161,162,163,164,165,166
0,230425980,-0.056189,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133295,-0.088508,...,-0.537739,-0.582752,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
1,5530458,-0.056193,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133311,-0.088517,...,1.016365,0.702410,-1.033609,-1.032884,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
2,232022460,-0.056378,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133972,-0.088883,...,0.731702,0.467009,-1.033609,-1.032884,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128
4,230460314,0.294595,-0.131268,-1.185821,0.125213,0.375512,0.188604,-0.101009,-0.134776,0.033618,...,-0.485972,-0.380435,0.484856,0.546219,0.003514,0.312670,0.369103,1.281230,0.201809,0.202687
5,230459870,0.279626,-0.131268,-1.185821,0.161872,0.375512,0.230500,-0.101009,-0.134832,0.031813,...,-0.478591,-0.402793,-0.270197,-0.160261,0.003514,0.312670,0.465697,1.137967,-0.689627,-0.689252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276696,-173005540,-0.028584,-0.153418,-1.185821,-0.168061,-0.046514,-0.146557,-0.101009,-0.101414,-0.052277,...,-0.499271,-0.500044,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
276698,-520729801,-0.026527,-0.061652,0.980956,-0.168061,-0.068726,-0.167505,-0.021890,-0.134233,-0.030002,...,-0.553127,-0.595477,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
276700,-80255194,-0.015471,0.043889,1.522650,-0.058083,-0.046514,-0.020871,-0.101009,-0.134902,-0.015154,...,-0.199222,-0.302816,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
276702,-223754303,-0.056336,-0.119014,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134890,-0.088828,...,-0.553127,-0.595477,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128


In [18]:
train_node_features_without_class = train_node_features.drop(columns=['class'])
train_node_features_without_class

,0,2,3,4,5,6,7,8,9,10,...,157,158,159,160,161,162,163,164,165,166
3,232438397,0.043005,0.743641,-0.644127,2.874647,-0.068726,2.304311,3.142835,-0.134902,-0.058112,...,-0.553127,-0.595477,0.204643,0.203652,0.994235,0.099367,-0.135784,0.669253,-0.102834,-0.102128
9,232029206,-0.006835,0.151139,-0.102433,0.913384,-0.068726,1.026508,0.136346,-0.134902,-0.074868,...,-0.553127,-0.595477,0.204643,0.203652,0.553915,0.014046,-0.135784,0.326888,-0.102834,-0.102128
10,232344069,-0.049186,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.108216,-0.074608,...,-0.553127,-0.595477,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
11,27553029,-0.050225,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.111938,-0.076671,...,-0.514659,-0.563666,-1.033609,-1.032884,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
16,3881097,-0.056437,-0.175567,-1.185821,-0.149731,-0.046514,-0.146557,-0.021890,-0.134896,-0.089198,...,-0.553127,-0.582752,0.204643,0.203652,0.003514,-0.071275,-0.084494,-0.058272,-1.775510,-1.775746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276695,-439441678,-0.047286,-0.163385,-0.644127,-0.186390,-0.068726,-0.167505,-0.101009,-0.101413,-0.070838,...,-0.553127,-0.582752,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
276697,-412642228,-0.026620,-0.118170,0.980956,-0.094742,-0.046514,-0.146557,0.136346,-0.128138,-0.063413,...,-0.553127,-0.582752,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128
276699,-79327033,-0.053926,-0.175567,-1.185821,-0.186390,-0.068726,-0.167505,-0.101009,-0.125191,-0.084016,...,-0.553127,-0.506406,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128
276701,-229310245,-0.056634,-0.141863,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134903,-0.089391,...,-0.553127,-0.570028,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128


In [19]:
# now we need to reindex the train nodes and train edges so that node id is equal to row id

reindex_dictionary = dict()
for i, node_id in enumerate(train_node_features_without_class[0]):
  # print(i)
  # print(node_id)
  # print()
  reindex_dictionary[node_id] = i
  # if i > 115: break

# reindex_dictionary

In [20]:
train_edges

,0,1
0,232344069,27553029
1,3881097,232457116
2,232051089,232470704
3,230473487,7089694
4,231182296,14660781
...,...,...
36628,-335492958,336712245
36629,-16565051,21440143
36630,-335493020,336712245
36631,-335493021,336712245


In [21]:
# reindex_dictionary

In [22]:
train_edges_reindexed = train_edges.copy()
train_edges_reindexed[0] = train_edges[0].map(lambda node_id: reindex_dictionary[node_id])
train_edges_reindexed[1] = train_edges[1].map(lambda node_id: reindex_dictionary[node_id])
train_edges_reindexed

,0,1
0,2,3
1,4,5
2,9,10
3,12,13
4,19,20
...,...,...
36628,48542,21892
36629,50245,5845
36630,58363,21892
36631,63973,21892


In [23]:
# now crafting the input data structures to pytorch geometric Data object
edge_index = torch.tensor(np.array(train_edges_reindexed), dtype=torch.long)
edge_index

tensor([[    2,     3],
        [    4,     5],
        [    9,    10],
        ...,
        [58363, 21892],
        [63973, 21892],
        [49121,  8639]])

In [24]:
x = torch.tensor(np.array(train_node_features_without_class), dtype=torch.float)

In [25]:
# train_classes.unique()

In [26]:

train_classes = train_node_features['class']
# y = torch.tensor(np.array(train_classes).reshape(len(train_classes), -1), dtype=torch.float)
y = torch.tensor(np.array(train_classes), dtype=torch.float)
y.shape

torch.Size([71496])

In [27]:
data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)
data.validate(raise_on_error = True)

True

In [28]:
data

Data(x=[71496, 166], edge_index=[2, 36633], y=[71496])

In [29]:
split = T.RandomNodeSplit(num_val=0.1, num_test=0)
data_with_masks = split(data)
data_with_masks





Data(x=[71496, 166], edge_index=[2, 36633], y=[71496], train_mask=[71496], val_mask=[71496], test_mask=[71496])

In [30]:
data.y

tensor([1., 1., 1.,  ..., 0., 0., 0.])

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree



In [32]:
# class Net(torch.nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
        
#         self.conv1 = GCNConv(166, 128)
#         self.conv4 = GCNConv(128, 1) 

#     def forward(self, data, adj=None):
#         x, edge_index = data.x, data.edge_index
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         x = F.dropout(x, p=0.1, training=self.training)
#         x = self.conv4(x, edge_index)

#         return F.sigmoid(x)
#         # return torch.sigmoid(x)

In [33]:
data_with_masks.num_features

166

In [48]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.hid = 8
        self.in_head = 8
        self.out_head = 1

        # self.conv1 = GATConv(data_with_masks.num_features, self.hid, heads = self.in_head, dropout = 0.6)
        # self.conv2 = GATConv(self.hid * self.in_head, 1, concat=False, heads = self.out_head, dropout = 0.6)
        self.conv1 = GATConv(data_with_masks.num_features, self.hid, heads = self.in_head)
        self.conv2 = GATConv(self.hid * self.in_head, 1, concat=False, heads = self.out_head)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training = self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training = self.training)
        x = self.conv2(x, edge_index)
        return F.sigmoid(x)

In [49]:
model = Net().to(device)
data_in_device = data_with_masks.to(device)

In [50]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
criterion = torch.nn.BCELoss()

In [51]:
# model.train()
# for epoch in range(170):
#     optimizer.zero_grad()
#     out = model(data_in_device.x, data_in_device.edge_index)
#     out = out.reshape((data.x.shape[0]))
#     loss = criterion(out[data.train_mask], data.y[data.train_mask])
#     auc = roc_auc_score(data.y.detach().cpu().numpy(), out.detach().cpu().numpy()) #[train_idx]
#     loss.backward()
#     optimizer.step()
#     if epoch%5 == 0:
        
#         print("epoch: {} - loss: {} - roc: {}".format(epoch, loss.item(), auc))

tensor([])

In [55]:
model.train()
for epoch in range(170):
    optimizer.zero_grad()
    out = model(data_in_device.x, data_in_device.edge_index)
    out = out.reshape((data.x.shape[0]))
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    auc = roc_auc_score(data.y.detach().cpu().numpy(), out.detach().cpu().numpy()) #[train_idx]

    if epoch%5 == 0:
        train_auc = roc_auc_score(data.y[data.train_mask].detach().cpu().numpy(), out[data.train_mask].detach().cpu().numpy()) 
        val_auc = roc_auc_score(data.y[data.val_mask].detach().cpu().numpy(), out[data.val_mask].detach().cpu().numpy())
        print("epoch: {} - loss: {} - train_roc: {} - val_auc: {}".format(epoch, loss.item(), train_auc, val_auc))
    loss.backward()
    optimizer.step()

epoch: 0 - loss: 39.38660430908203 - train_roc: 0.3022546649747162 - val_auc: 0.30597149993168327
epoch: 5 - loss: 39.66862869262695 - train_roc: 0.3730220391164177 - val_auc: 0.37581231378826474
epoch: 10 - loss: 34.238861083984375 - train_roc: 0.44347660061643485 - val_auc: 0.4371829017721562
epoch: 15 - loss: 27.23910140991211 - train_roc: 0.5344460732285407 - val_auc: 0.5400841704564661
epoch: 20 - loss: 18.019439697265625 - train_roc: 0.6519783349344404 - val_auc: 0.6520865427640249
epoch: 25 - loss: 12.76635456085205 - train_roc: 0.7266546773090525 - val_auc: 0.7184220755084741
epoch: 30 - loss: 9.263813972473145 - train_roc: 0.7842956659211608 - val_auc: 0.7785543008152035
epoch: 35 - loss: 7.805080890655518 - train_roc: 0.8108333045477345 - val_auc: 0.815508041951023
epoch: 40 - loss: 7.275641441345215 - train_roc: 0.8207607792376945 - val_auc: 0.813844399353225
epoch: 45 - loss: 9.573902130126953 - train_roc: 0.8096601594871149 - val_auc: 0.8078118433415767
epoch: 50 - loss: 8

KeyboardInterrupt: 